In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/Users/waldosingleton/Desktop/OneDrive - The University of Auckland/3rd Year/361/ML-for-Social-Good---Group-1/heart_disease_health_indicators_BRFSS2015.csv", skiprows=0, header=0, na_values= "", dtype=str)

df = df.sample(frac=1)

disease = df.loc[df['HeartDiseaseorAttack'] == '1.0'][:25000]
no_disease = df.loc[df['HeartDiseaseorAttack'] == '0.0'][:25000]

balanced_data = pd.concat([disease, no_disease])
balanced_data = balanced_data.sample(frac=1)

balanced_data.head()



,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
81033,0.0,1.0,1.0,1.0,40.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,5.0,5.0
85374,0.0,1.0,1.0,1.0,34.0,1.0,0.0,2.0,0.0,1.0,...,1.0,0.0,4.0,30.0,30.0,0.0,0.0,3.0,3.0,4.0
248462,0.0,0.0,1.0,1.0,34.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,30.0,0.0,1.0,5.0,4.0,8.0
109345,1.0,1.0,1.0,1.0,28.0,1.0,1.0,2.0,0.0,0.0,...,1.0,0.0,3.0,0.0,4.0,0.0,1.0,10.0,4.0,6.0
140742,1.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,4.0,0.0,0.0,0.0,1.0,2.0,6.0,6.0
